In [1]:
!pip install chromedriver-autoinstaller --trusted-host pypi.org --trusted-host files.pythonhosted.org --no-cache-dir

Defaulting to user installation because normal site-packages is not writeable


### 1. 기본 셀레니움 코드

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# ChromeDriver 경로 설정
driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"  # 사용자 환경에 맞게 경로 수정
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 로그인 페이지로 이동
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
driver.get(login_url)

# 입력 필드 찾기
username_field = driver.find_element(By.ID, "login_id")  
password_field = driver.find_element(By.ID, "login_password")

# 로그인 정보 입력(임시 비밀번호)
username_field.send_keys("hoykim125")
password_field.send_keys("hoykim125!!")

# 로그인 버튼 클릭
login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button') # 버튼 속성에 맞게 변경
login_button.click()

# 원하는 페이지로 이동
driver.get("https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0000078229")

# 페이지 소스 가져오기
page_source = driver.page_source

# 해당 페이지에서 시험 데이터 크롤링
company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

# 결과 출력
print("기업명:", company_info_name)
print("대표자명:", representative_name)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td[headers='row01']"}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001028abb08 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001028a4358 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x000000010230c4b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000102350898 cxxbridge1$string$len + 368748
4   chromedriver                        0x000000010238a0fc cxxbridge1$string$len + 604368
5   chromedriver                        0x00000001023450b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000102345d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x

1 코드 결과: "500 연결하려는 페이지에 문제가 있어서 페이지를 표시할 수 없습니다. 잠시 후 다시 사용하여 주시기 바랍니다. 이용에 불편을 드려 대단히 죄송합니다.." 표시됨

### 2. request 추가

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests

# User-Agent 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.140 Safari/537.36"

# ChromeDriver 옵션 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(f"user-agent={user_agent}")

# ChromeDriver 경로 설정
driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"  # 사용자 환경에 맞게 경로 수정
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# 로그인 URL
login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"

# 로그인 데이터 설정
login_data = {
    "login_id": "hoykim125",  # 사용자 ID
    "login_password": "hoykim125!!"  # 사용자 비밀번호
}

# Requests로 로그인 페이지 요청 (헤더에 User-Agent 추가)
headers = {"User-Agent": user_agent}
session = requests.Session()
response = session.post(login_url, data=login_data, headers=headers)

# 로그인 세션 공유를 위해 Selenium 실행
try:
    # Selenium으로 로그인 페이지 이동
    driver.get(login_url)

    # 입력 필드 찾기
    username_field = driver.find_element(By.ID, "login_id")
    password_field = driver.find_element(By.ID, "login_password")

    # 로그인 정보 입력
    username_field.send_keys(login_data["login_id"])
    password_field.send_keys(login_data["login_password"])

    # 로그인 버튼 클릭
    login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')
    login_button.click()

    # 로그인 후 타겟 페이지로 이동
    target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0000078229"
    driver.get(target_url)

    # 해당 페이지에서 데이터 크롤링
    company_info_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row01']").text
    representative_name = driver.find_element(By.CSS_SELECTOR, "td[headers='row02']").text

    # 결과 출력
    print("기업명:", company_info_name)
    print("대표자명:", representative_name)

except Exception as e:
    print("오류 발생:", e)

finally:
    # 브라우저 닫기
    driver.quit()

SSLError: HTTPSConnectionPool(host='sminfo.mss.go.kr', port=443): Max retries exceeded with url: /cm/sv/CSV001R0.do (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))

2 코드결과: Chrome이 자동화된 테스트 소프트웨어에 의해 제어되고 있습니다. 라는 메세지와 함께 크롬 빈 화면이 뜹니다.

### 3. (오류에 대한 피드백에 따라)request관련 코드 제거 및 excludeSwitches 등 자동화 탐지 방지 코드 추가

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# User-Agent 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.140 Safari/537.36"

# Selenium 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument(f"user-agent={user_agent}")

# ChromeDriver 경로 설정
driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 로그인 페이지 접속
    login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
    driver.get(login_url)

    # 로그인 입력 필드 찾기 및 입력
    username_field = driver.find_element(By.ID, "login_id")
    password_field = driver.find_element(By.ID, "login_password")
    username_field.send_keys("hoykim125")
    password_field.send_keys("hoykim125@@")

    # 로그인 버튼 클릭
    login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')
    login_button.click()

    # 타겟 페이지로 이동
    target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0007901371"
    driver.get(target_url)

    # 데이터 크롤링
    company_info_name = driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[1]').text
    representative_name = driver.find_element(By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[2]').text

    # 결과 출력
    print("기업명:", company_info_name)
    print("대표자명:", representative_name)

except Exception as e:
    print("오류 발생:", e)

finally:
    # 브라우저 닫기
    driver.quit()

오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[1]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001027b3b08 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001027ac358 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x00000001022144b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000102258898 cxxbridge1$string$len + 368748
4   chromedriver                        0x00000001022920fc cxxbridge1$string$len + 604368
5   chromedriver                        0x000000010224d0b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x000000010224dd00 cxxbridge1$string$len + 324820
7   chromedriver           

## 4. requests 재추가 및 Xpath 주소 재확인

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

# User-Agent 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.140 Safari/537.36"

# Selenium 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument(f"user-agent={user_agent}")

driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 1. Selenium으로 로그인 처리
    login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
    driver.get(login_url)

    username_field = driver.find_element(By.ID, "login_id")
    password_field = driver.find_element(By.ID, "login_password")
    username_field.send_keys("hoykim125")
    password_field.send_keys("hoykim125@@")

    login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')
    login_button.click()

    # 로그인 후 쿠키 가져오기
    cookies = driver.get_cookies()

    # 2. Requests 세션에 쿠키 설정
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # Requests 헤더 설정
    headers = {"User-Agent": user_agent}

    # 3. Requests로 타겟 페이지 요청
    target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0007901371"
    response = session.get(target_url, headers=headers, verify=False)

    # 4. HTML에서 데이터 추출
    if response.status_code == 200:
        print("Requests로 데이터를 성공적으로 가져왔습니다.")
        print(response.text)  # HTML 전체 출력 
    else:
        print(f"요청 실패: 상태 코드 {response.status_code}")

except Exception as e:
    print("오류 발생:", e)

finally:
    driver.quit()

/Users/kimble/Library/Python/3.12/lib/python/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sminfo.mss.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


요청 실패: 상태 코드 500


## 5. selenium과 request 모두 이용 

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests

# User-Agent 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.140 Safari/537.36"

# Selenium 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument(f"user-agent={user_agent}")

driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 1. Selenium으로 로그인 처리
    login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
    driver.get(login_url)

    username_field = driver.find_element(By.ID, "login_id")
    password_field = driver.find_element(By.ID, "login_password")
    username_field.send_keys("hoykim125")
    password_field.send_keys("hoykim125@@")

    login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')
    login_button.click()

    # 로그인 후 쿠키 가져오기
    cookies = driver.get_cookies()

    # Selenium으로 타겟 데이터 크롤링
    target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0007901371"
    driver.get(target_url)

    # 데이터 크롤링 (WebDriverWait 사용)
    wait = WebDriverWait(driver, 10)
    company_info_name = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[1]'))
    ).text
    representative_name = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[2]'))
    ).text

    # 결과 출력 (Selenium)
    print("Selenium 크롤링 결과:")
    print("기업명:", company_info_name)
    print("대표자명:", representative_name)

    # 2. Requests 세션에 쿠키 설정
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # Requests 헤더 설정
    headers = {"User-Agent": user_agent}

    # Requests로 타겟 페이지 요청
    response = session.get(target_url, headers=headers, verify=False)

    # BeautifulSoup으로 데이터 파싱
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        company_info_name_req = soup.select_one('td[headers="row01"]').get_text(strip=True)
        representative_name_req = soup.select_one('td[headers="row02"]').get_text(strip=True)

        # 결과 출력 (Requests)
        print("\nRequests 크롤링 결과:")
        print("기업명:", company_info_name_req)
        print("대표자명:", representative_name_req)
    else:
        print(f"Requests 요청 실패: 상태 코드 {response.status_code}")

except Exception as e:
    print("오류 발생:", e)

finally:
    # 브라우저 닫기
    driver.quit()

오류 발생: Message: 
Stacktrace:
0   chromedriver                        0x0000000102e53b08 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000102e4c358 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x00000001028b44b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x00000001028f8898 cxxbridge1$string$len + 368748
4   chromedriver                        0x00000001029320fc cxxbridge1$string$len + 604368
5   chromedriver                        0x00000001028ed0b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x00000001028edd00 cxxbridge1$string$len + 324820
7   chromedriver                        0x0000000102e1ee4c cxxbridge1$str$ptr + 3435328
8   chromedriver                        0x0000000102e22164 cxxbridge1$str$ptr + 3448408
9   chromedriver                        0x0000000102e061c0 cxxbridge1$str$ptr + 3333812
10  chromedriver                        0x0000000102e22a24 cxxbridge1$str$ptr + 34

## 6. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

# User-Agent 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.140 Safari/537.36"

# Selenium 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument(f"user-agent={user_agent}")

driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 1. Selenium으로 로그인 처리
    login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
    driver.get(login_url)

    username_field = driver.find_element(By.ID, "login_id")
    password_field = driver.find_element(By.ID, "login_password")
    username_field.send_keys("hoykim125")
    password_field.send_keys("hoykim125@@")

    login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')
    login_button.click()

    # 로그인 후 쿠키 가져오기
    cookies = driver.get_cookies()

    # 2. Requests 세션에 쿠키 설정
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # Requests 헤더 설정
    headers = {"User-Agent": user_agent}

    # 3. Requests로 타겟 페이지 요청
    target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0007901371"
    response = session.get(target_url, headers=headers, verify=False)

    # 4. HTML에서 데이터 추출
    if response.status_code == 200:
        print("Requests로 데이터를 성공적으로 가져왔습니다.")
        print(response.text)  # HTML 전체 출력 
    else:
        print(f"요청 실패: 상태 코드 {response.status_code}")

except Exception as e:
    print("오류 발생:", e)

finally:
    driver.quit()

/Users/kimble/Library/Python/3.12/lib/python/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sminfo.mss.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


요청 실패: 상태 코드 500


## 7. SSL 인증서와 관련해서 requests는 기본적으로 SSL 인증서를 검증하므로 verify 매개변수를 생략

In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import urllib3 #인증서 관련 경고 무시 
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) 

# User-Agent 설정
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.6778.140 Safari/537.36"

# Selenium 설정
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument(f"user-agent={user_agent}")

driver_path = "/Users/kimble/Downloads/chromedriver-mac-arm64/chromedriver"
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

try:
    # 1. Selenium으로 로그인 처리
    login_url = "https://sminfo.mss.go.kr/cm/sv/CSV001R0.do"
    driver.get(login_url)

    username_field = driver.find_element(By.ID, "login_id")
    password_field = driver.find_element(By.ID, "login_password")
    username_field.send_keys("hoykim125")
    password_field.send_keys("hoykim125@@")

    login_button = driver.find_element(By.XPATH, '//*[@id="main_link"]/div/div[1]/div/form/fieldset/div/button')
    login_button.click()

    # 로그인 후 쿠키 가져오기
    cookies = driver.get_cookies()

    # Selenium으로 타겟 데이터 크롤링
    target_url = "https://sminfo.mss.go.kr/si/ei/IEI001R0.do?cmd=com&kcd=0007901371"
    driver.get(target_url)

    # 데이터 크롤링 (WebDriverWait 사용)
    wait = WebDriverWait(driver, 10)
    company_info_name = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[1]'))
    ).text
    representative_name = wait.until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="contents_sub"]/div[3]/div/table/tbody/tr[1]/td[2]'))
    ).text

    # 결과 출력 (Selenium)
    print("Selenium 크롤링 결과:")
    print("기업명:", company_info_name)
    print("대표자명:", representative_name)

    # 2. Requests 세션에 쿠키 설정
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # Requests 헤더 설정
    headers = {"User-Agent": user_agent}

    # Requests로 타겟 페이지 요청
    response = session.get(target_url, headers=headers, verify=False)

    # BeautifulSoup으로 데이터 파싱
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        company_info_name_req = soup.select_one('td[headers="row01"]').get_text(strip=True)
        representative_name_req = soup.select_one('td[headers="row02"]').get_text(strip=True)

        # 결과 출력 (Requests)
        print("\nRequests 크롤링 결과:")
        print("기업명:", company_info_name_req)
        print("대표자명:", representative_name_req)
    else:
        print(f"Requests 요청 실패: 상태 코드 {response.status_code}")

except Exception as e:
    print("오류 발생:", e)

finally:
    # 브라우저 닫기
    driver.quit()

오류 발생: Message: 
Stacktrace:
0   chromedriver                        0x0000000101303b08 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001012fc358 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x0000000100d644b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000100da8898 cxxbridge1$string$len + 368748
4   chromedriver                        0x0000000100de20fc cxxbridge1$string$len + 604368
5   chromedriver                        0x0000000100d9d0b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000100d9dd00 cxxbridge1$string$len + 324820
7   chromedriver                        0x00000001012cee4c cxxbridge1$str$ptr + 3435328
8   chromedriver                        0x00000001012d2164 cxxbridge1$str$ptr + 3448408
9   chromedriver                        0x00000001012b61c0 cxxbridge1$str$ptr + 3333812
10  chromedriver                        0x00000001012d2a24 cxxbridge1$str$ptr + 34

7 코드 결과: "500 연결하려는 페이지에 문제가 있어서 페이지를 표시할 수 없습니다. 잠시 후 다시 사용하여 주시기 바랍니다. 이용에 불편을 드려 대단히 죄송합니다.." 표시됨